# Trim Primers

Given a single-sequence FASTQ file and primer ID, updates the file to contain only the sequence between the primers.

## Setup

In [2]:
%load_ext autoreload
%autoreload 1
%aimport RCUtils

In [32]:
from Bio import Align
from Bio import SeqIO
import matplotlib_inline.backend_inline
import os
import pandas as pd
import RCUtils
import random
import glob

qPCRprimers = RCUtils.readPrimers("HRVPrimers.fasta", display=True)
RCprimers = RCUtils.readPrimers("RespiCovPrimers.fasta", display=False)

def getPrimers(name):
    return filter(lambda p: p.name.startswith(name), qPCRprimers)

def getRCprimers(name):
    return filter(lambda p: p.description.startswith(name), RCprimers)

from pathlib import Path

def trimAllPrimers(pathGlob,primers):
    for path in glob.glob(pathGlob):
        trimPrimers(path, primers)

def trimPrimers(fastQPath, primers):
    record = SeqIO.read(fastQPath, "fastq")
    hits = RCUtils.computePrimerHits(record, primers)
    print()
    print(f"{fastQPath}, len={len(record.seq)}")
    last_forward = None
    first_reverse = None
    for i, hit in enumerate(hits):
        r = "(rev) " if hit.rev else ""
        print(f" {hit.primer.description} {r}{hit.start}-{hit.end} ({hit.mr*100:.0f}%)")
        if not hit.rev:
            last_forward = i
        if hit.rev and first_reverse is None and last_forward is not None:
            first_reverse = i

    if last_forward is None:
        print("  ERROR: no forward primer")
    elif first_reverse is None:
        print("  ERROR: no reverse primer")
    elif last_forward >= first_reverse:
        print("  ERROR: hits in wrong order")
    else:
        start = hits[last_forward].end
        end = hits[first_reverse].start
        trimmed = record[start:end]
        print(f" trimmed to {start}-{end} len={len(trimmed.seq)}")
        SeqIO.write(trimmed, fastQPath+".trimmed", "fastq")
    

Reading primers: HRVPrimers.fasta
  ENTng-f (2 variations)
  ENTng-r
  ENTng-p (8 variations)
  ENTrc-f1
  ENTrc-f2
  ENTrc-r
  HRVma-f
  HRVma-r
  HRVma-p
  HRVkaV-fo (2 variations)
  HRVkaV-fi
  HRVkaV-r (768 variations)
  HRVka5-f
  HRVka5-ro
  HRVka5-ri
  HRVbo-f (4 variations)
  HRVbo-r
  HRVbo-p
Read 797 primers


In [27]:
trimPrimers("myseqs/S28-RVA-23.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S44-RVA-56.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S48-RVC-1.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S59-RVA-77.fastq", getPrimers("HRVka5"))



myseqs/S28-RVA-23.fastq, len=392
 ENTrc-f1 1-22 (100%)
 ENTrc-r 370-391 (100%)
 trimmed to 22-370 len=348

myseqs/S44-RVA-56.fastq, len=417
 ENTrc-f1 23-44 (95%)
 ENTrc-r 394-415 (100%)
 trimmed to 44-394 len=350

myseqs/S48-RVC-1.fastq, len=397
 ENTrc-f1 1-22 (100%)
 ENTrc-r 374-395 (100%)
 trimmed to 22-374 len=352

myseqs/S59-RVA-77.fastq, len=437
 HRVka5-f 16-37 (100%)
 HRVka5-ri 291-309 (100%)
 HRVka5-ro 388-411 (100%)
 trimmed to 37-388 len=351


In [ ]:
trimPrimers("myseqs/S142-RVA-62.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S147-RVC-44.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S148-RVA-68.fastq", getPrimers("ENTrc"))
trimPrimers("myseqs/S153-RVB-27.fastq", getPrimers("ENTrc"))



myseqs/S142-RVA-62.fastq, len=346
  ERROR: didn't get 2 primer hits

myseqs/S147-RVC-44.fastq, len=338
  ERROR: didn't get 2 primer hits

myseqs/S148-RVA-68.fastq, len=349
  ERROR: didn't get 2 primer hits

myseqs/S153-RVB-27.fastq, len=333
 HRVka5-ri 60-78 (100%)
  ERROR: didn't get 2 primer hits


In [22]:
trimPrimers("myseqs/S134-PIV3.fastq", getRCprimers("Respirovirus"))


myseqs/S134-PIV3.fastq, len=420
 Respirovirus 5 27-49 (100%)
 Respirovirus 6 377-400 (100%)
 trimmed to 49-377 len=328


In [33]:
trimPrimers("myseqs/S121-OC43.fastq", getRCprimers("Human coronavirus OC43"))


myseqs/S121-OC43.fastq, len=11012
 Human coronavirus OC43 1 (rev) 10648-10667 (82%)
 Human coronavirus OC43 1 10669-10691 (100%)
 Human coronavirus OC43 2 (rev) 10967-10988 (100%)
 trimmed to 10691-10967 len=276
